In [6]:
import lxml.html as html
import io
import nltk as nl
import string
import math
import newspaper
from newspaper import Article

In [28]:
mk_paper = newspaper.build(u'http://www.mk.ru/')

CRITICAL:newspaper.network:[REQUEST FAILED] (<requests.packages.urllib3.connectionpool.HTTPConnectionPool object at 0x00000000FD3390B8>, 'Connection to fregat.com.ru timed out. (connect timeout=7)')


In [3]:
def get_corpus(category,num_texts):
    num = 0
    all_text = ""
    for article in mk_paper.articles:
        if num == num_texts:
            break
        if article.url.find(category) != -1:
            tmp = Article(article.url)
            tmp.download()
            tmp.parse()
            all_text += tmp.text + '\n'
            num += 1
    return all_text
def get_grams(file):
    f = io.open(file, "r", encoding='utf8')
    n_dict = {}
    for line in f:
        line = line.replace('\n', '').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    f.close()
    return n_dict

def prepare(cat_text):
    ftext = []
    for fline in cat_text:
        ftext = ftext + nl.word_tokenize(fline)
        for item in ftext:
            if item in string.punctuation:
                ftext.remove(item)
    return ftext

def calc_perp(text, tgram, bgram):
    sum_prob = 0
    uwords = 0
    curr_prob= {}
    min_prob = 1
    for i in range(2, len(text)):
        try:
            curr_prob[i] = (tgram[(text[i-2], text[i-1], text[i])] + 0.0) / bgram[(text[i-2],text[i-1])]
            sum_prob += curr_prob
            if curr_prob[i] < min_prob:
                min_prob = curr_prob[i]
        except KeyError:
            uwords +=1
    kwords = len(text) - uwords
    min_prob /= 10.0
    psub = (uwords * min_prob + 0.0) / kwords  
    sum = 0
    for i in range(2, len(text)):
        try:
            sum += math.log(curr_prob[i] - psub, 2)
        except KeyError:
            sum += math.log(min_prob, 2)

    return 2**(-sum/len(text))

sports =  get_corpus('www.mk.ru/sport',3)
politics = get_corpus('www.mk.ru/politics',3)

CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects


In [9]:
b_grams = load_ngrams('2grams.txt')
t_grams = load_ngrams('3grams.txt')

In [26]:
print "Sports perplexity   = ",calc_perp(prepare(sports), b_grams, t_grams)
print "Politics perplexity = ",calc_perp(prepare(politics), b_grams, t_grams)

Sports perplexity   =  9.95918502855
Politics perplexity =  9.96275050375
